In [1]:
#THINGS TO CHANGE BEFORE NEXT SAMPLE:

#1. PATH TO TXT 
#2. PATHS IN MIDDLE
#3. FINAL TABLE NAME 

##CREATE TABLE FROM ISOSEQ DATA 


import pandas as pd
import glob

##OPEN ISOSEQ DATA AND RECORD SPECIFIC COLUMNS
##MUST MAKE PATH TO FILE THE TYPE OF TISSUE

iso_seq = pd.read_csv("ah1SNP.txt", sep='\t', 
            names=["chr","pos","ref","depth","Nucs", "Quality"])

##PUT NUC.S INTO LIST 

nucs = iso_seq["Nucs"]
nucs_dict = nucs.to_dict()
nucs_list = list(nucs_dict.values())

x = 0

##APPENDABLE LISTS FOR EACH NUC.

A = []
C = []
G = []
T = []

##COUNT NUMBER OF NUC.S PER ROW 

while x < iso_seq.shape[0]:
    
    A_count = nucs_list[x].count('A')
    C_count = nucs_list[x].count('C')
    G_count = nucs_list[x].count('G')
    T_count = nucs_list[x].count('T')
    a_count = nucs_list[x].count('a')
    c_count = nucs_list[x].count('c')
    g_count = nucs_list[x].count('g')
    t_count = nucs_list[x].count('t')
    
    a = A_count + a_count 
    c = C_count + c_count 
    g = G_count + g_count 
    t = T_count + t_count 

##MAKE READS OF ZERO == 1, (for binom dist)

    if (a+c+g+t) == 0:
        t = 1
    
    A.append(a)
    C.append(c)
    G.append(g)
    T.append(t) 
    
    x += 1

##APPEND THE LISTS OF NUC.S TO DF
iso_seq['A'] = A
iso_seq['C'] = C
iso_seq['G'] = G
iso_seq['T'] = T

##SEND DF TO FARM TO CHECK 
#iso_seq.to_csv('Alleles.csv')
iso_seq

,chr,pos,ref,depth,Nucs,Quality,A,C,G,T
0,chr1,1830540,N,281,TTGTTTTTTGTTTGTTGTGTTTTTTGTGTGGTGTTGTGTTGTGGTG...,:kkkkFFkFFFFkFkkFFFFFkFFkFFF:FkkFkFFkFFkkFFFkF...,0,0,115,166
1,chr1,1843985,N,366,CTC$C$C$cctcttccccctctttcttctttccctttcctctcTCC...,FFkkkFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFkkkFFk...,0,188,0,178
2,chr1,3451066,N,17,>GGGGTGGGGG^~G^~G^~G^~G^~G^~G,:FkFFFkFFFkk_FkFk,0,0,15,1
3,chr1,8675856,N,82,GaaGaggGgagaAaaAAGGggaGAgaaGaaaaagagGGAGAaGggG...,FF::FFFFFFFFFFFFFFFFFFkFFFFFFFFFFFFFFkFFFFFFFF...,43,0,39,0
4,chr1,8680767,N,70,g$gAAGAAaAAAaAGaggaGaGgGAaggaGAAagaaAGgGGAAGGA...,FFkkkkkFkkkFkkFFFF_F_F:_FFFFFFkFFFFFFFkkkk:kFk...,41,0,29,0
...,...,...,...,...,...,...,...,...,...,...
1941,chrX,9658663,N,1204,GGg><g$g$g$g$g$g$g$ggGGGG$GgggGGggGGGggggggGGG...,FFFFFFFFFFFFFFkk_QkFFFkkFFkQkFFFFFFkFkkk_kkFF:...,0,0,1202,0
1942,chrX,9658668,N,1138,AAa><aaaAAaAAAAAAAAAAAaAAaaAaaaaAaaaaaaaaaaaaa...,FFFFFFFFk_Fk:7kkkk_Qk_FQk:FkFFFFkFFFFFFFFFFFFF...,1136,0,0,0
1943,chrX,9658756,N,777,Ggg$g$g$g$g$g$g$g$g$G$g$g$g$g$g$g$g$g$g$g$g$g$...,FFFFFFFFFFFkFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFkFF...,0,0,777,0
1944,chrX,9658813,N,345,gg$g$G$gggggggggggggggggggG$gggggggggggggggggg...,FFF_FFFFFFFFFFFFFFFFFF:kFFFFFFFFFFFFFFFFFFFFFF...,0,0,345,0


In [2]:
##INPUT TRANSCRIPT ID INTO TABLE 

##TRANSCRIPT ID'S FOUND WITHIN FARM (PHASED NO PARTIAL CLEANED VCF)

##CREATE A NEW DATAFRAME 

df = pd.DataFrame(columns = ['chr', 'pos', 'pbID'])

##TAKE ELEMENTS FROM FILES WITHIN FARM

for file in glob.glob("/group/ctbrowngrp/finnolab_shared/phased_isoseq/AH1_Liver/*/phased.nopartial.cleaned.vcf"):

##split the file on '/', take the [6] element 
##further split the file, take the [0] element 
    
    ID = file.split('/')[6].split('_')[0]

##open the created file, leave out comments, seperate by tab

    var_df = pd.read_csv(file, comment ="#", sep = '\t', header = None)

##slice list for chromosone # and position

    var_df = var_df.iloc[:,[0,1]]
    var_df.columns = ['chr', 'pos']

##create a new column for TranscriptID, fill with data 

    var_df['pbID'] = ID
    
##combine both columns 
    df = pd.concat([var_df, df], axis = 0)

##Merge the first data frame (vr) with new df

merged_df = pd.merge(iso_seq,df,how ='left', on=None, left_on=['chr','pos'], right_on=['chr','pos'],
)

##SEND DF TO FARM TO CHECK 

#merged_df.to_csv("merged_df.csv")

merged_df = merged_df.dropna()

In [3]:
##GET HAPLOTYPES (TOGETHER) 

#import pandas as pd 
#import glob 

df = pd.DataFrame(columns = ['pbID'])
 
for file in glob.glob("/group/ctbrowngrp/finnolab_shared/phased_isoseq/AH1_Liver/*/phased.nopartial.cleaned.human_readable.txt"):

##split the file on '/', take the [6] element 
##further split the file, take the [0] element 
    
    ID = file.split('/')[6].split('_')[0]

##open the created file, leave out comments, seperate by tab

    var_df = pd.read_csv(file, sep = '\t', header = None, skiprows = 2)

##slice list for chromosone # and position

    var_df = var_df.iloc[:,[0]]
    
    var_df.columns = ['Haps']

##create a new column for TranscriptID, fill with data 

    var_df['pbID'] = ID
    
##COMBINE LISTS INTO DF 

    df = pd.concat([var_df, df], axis = 0)


##MERGE DATAFRAMES TOGETHER 

merged_df2 = pd.merge(merged_df,df,how ='left', on=None, left_on=['pbID'], right_on=['pbID'],
)

#SEND DF TO FARM TO CHECK 

#merged_df2.to_csv("merged_df2.csv")
#merged_df2

In [4]:
merged_df2.head(15)

,chr,pos,ref,depth,Nucs,Quality,A,C,G,T,pbID,Haps
0,chr1,1830540,N,281,TTGTTTTTTGTTTGTTGTGTTTTTTGTGTGGTGTTGTGTTGTGGTG...,:kkkkFFkFFFFkFkkFFFFFkFFkFFF:FkkFkFFkFFkkFFFkF...,0,0,115,166,PB.14,TC
1,chr1,1830540,N,281,TTGTTTTTTGTTTGTTGTGTTTTTTGTGTGGTGTTGTGTTGTGGTG...,:kkkkFFkFFFFkFkkFFFFFkFFkFFF:FkkFkFFkFFkkFFFkF...,0,0,115,166,PB.14,GT
2,chr1,1843985,N,366,CTC$C$C$cctcttccccctctttcttctttccctttcctctcTCC...,FFkkkFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFkkkFFk...,0,188,0,178,PB.14,TC
3,chr1,1843985,N,366,CTC$C$C$cctcttccccctctttcttctttccctttcctctcTCC...,FFkkkFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFkkkFFk...,0,188,0,178,PB.14,GT
4,chr1,3451066,N,17,>GGGGTGGGGG^~G^~G^~G^~G^~G^~G,:FkFFFkFFFkk_FkFk,0,0,15,1,PB.915,G
5,chr1,3451066,N,17,>GGGGTGGGGG^~G^~G^~G^~G^~G^~G,:FkFFFkFFFkk_FkFk,0,0,15,1,PB.915,T
6,chr1,8675856,N,82,GaaGaggGgagaAaaAAGGggaGAgaaGaaaaagagGGAGAaGggG...,FF::FFFFFFFFFFFFFFFFFFkFFFFFFFFFFFFFFkFFFFFFFF...,43,0,39,0,PB.34,AA
7,chr1,8675856,N,82,GaaGaggGgagaAaaAAGGggaGAgaaGaaaaagagGGAGAaGggG...,FF::FFFFFFFFFFFFFFFFFFkFFFFFFFFFFFFFFkFFFFFFFF...,43,0,39,0,PB.34,GG
8,chr1,8680767,N,70,g$gAAGAAaAAAaAGaggaGaGgGAaggaGAAagaaAGgGGAAGGA...,FFkkkkkFkkkFkkFFFF_F_F:_FFFFFFkFFFFFFFkkkk:kFk...,41,0,29,0,PB.34,AA
9,chr1,8680767,N,70,g$gAAGAAaAAAaAGaggaGaGgGAaggaGAAagaaAGgGGAAGGA...,FFkkkkkFkkkFkkFFFF_F_F:_FFFFFFkFFFFFFFkkkk:kFk...,41,0,29,0,PB.34,GG


In [5]:
##SEPERATE HAPLOTYPES INTO SEPERATE COLUMNS

#import pandas 
#import glob 

##GET HAPLOTYPE 1

h1 = merged_df2[['pbID','Haps']]

##TAKE EVERY SECOND COLUMN STARTING AT 0

h1 = h1.iloc[::2, :]
h1.rename(columns = {'Haps':'Hap1'}, inplace = True)

##GET HAPLOTYPE 2

h2 = merged_df2[['pbID','Haps']]

##TAKE EVERY SECOND COLUMN STARTING AT 1

h2 = h2.iloc[1::2, :]
h2.rename(columns = {'Haps':'Hap2'}, inplace = True)

##MERGE BOTH LISTS TO MAIN DATA FRAME

new_list = pd.merge(merged_df2,h1,how ='left', on=None, left_on=['pbID'], right_on=['pbID'],
)
new_list2 = pd.merge(new_list,h2,how ='left', on=None, left_on=['pbID'], right_on=['pbID'],
)

##CREATE NEW TABLE WITH NO DUPLICATE ROWS 

new_list3 = new_list2.drop(['Haps'], axis = 1 )

merged_df3 = new_list3.drop_duplicates()
 
#new_list3

#SEND DF TO FARM TO CHECK 

#merged_df3.head(10)
#merged_df3.to_csv("merged_df3.csv")


In [17]:
#merged_df3.head(10)
merged_df3.loc[merged_df3['pbID'] == 'PB.1687']

,chr,pos,ref,depth,Nucs,Quality,A,C,G,T,pbID,Hap1,Hap2
7478,chr1,163304124,N,6631,>>><>><><><<<<<<>A>AaA>><a<<<<><>><G>>>>>>>>>>...,FF:::F:FFF:FFFFFFFFF:FFFFFFFFFFFF:FFFFFFFFFFFF...,2945,0,2726,0,PB.1687,GCCCGAGC,ATTTAGAT
7606,chr1,163305087,N,4412,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><>>>>>...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,0,1318,0,2160,PB.1687,GCCCGAGC,ATTTAGAT
7734,chr1,163305090,N,4376,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><>>>>>...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,0,1305,0,2137,PB.1687,GCCCGAGC,ATTTAGAT
7862,chr1,163307165,N,5606,>>>>><>><>>>><>><>><>><><>><>><>><>><><>><><>>...,FFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFF...,2,2388,1,2610,PB.1687,GCCCGAGC,ATTTAGAT
7990,chr1,163308852,N,4175,>>><>><>><>><><>><><>><><><><><><><<>><><>><><...,F:FFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFF...,1952,0,1650,1,PB.1687,GCCCGAGC,ATTTAGAT
8118,chr1,163313242,N,6483,>A>G>>G<><>>><>><>><>><>><><>><>>><><><>><>><>...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,3066,0,3106,0,PB.1687,GCCCGAGC,ATTTAGAT
8246,chr1,163320831,N,5990,>><>><><>><>>><><><>><>><>><>><><>>><><><><><>...,FFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFF...,2948,2,2753,0,PB.1687,GCCCGAGC,ATTTAGAT
8374,chr1,163322342,N,4993,><><CctT$T$T$T$T$T$T$t$T$T$T$t$T$t$t$T$t$t$T$T...,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,0,2516,0,2473,PB.1687,GCCCGAGC,ATTTAGAT


In [7]:
#parse.groupby('pbID').apply(lambda df: df.shape[0] == df['nallele'].iloc[0]).index

In [8]:
#parse.apply(pd.to_numeric, errors="ignore").applymap(lambda x: isinstance(x, float), na_action='ignore').any()

In [9]:
parse = merged_df3[['pbID','A','C','G','T','Hap1','Hap2']]
parse = parse.dropna()
parse['nallele']=parse['Hap1'].astype('str').apply(len)
fids = parse.groupby('pbID').apply(lambda df: df.shape[0] == df['nallele'].iloc[0])[~parse.groupby('pbID').apply(lambda df: df.shape[0] == df['nallele'].iloc[0])]
parse = parse[~parse['pbID'].isin(fids.index)]

def parse_haplotype(table):
    
     # Get haplotypes aligned with loci
     # Give following format:
     # gene  Hap1  Hap2
     # PB1  C  A
     # PB1  T  C
     # PB1  T  G
        
    table['Hap1']   = list(table['Hap1'].iloc[0])
    
    table['Hap2']   = list(table['Hap2'].iloc[0])
     
    # Get read count based on variant at each locus
     
    table['hap1_n'] = table.apply(lambda row: row.loc[row.loc['Hap1']], axis = 1)
    
    table['hap2_n'] = table.apply(lambda row: row.loc[row.loc['Hap2']], axis = 1)
    
    return table

z = parse.groupby('pbID').apply(parse_haplotype)
z = z.groupby('pbID').sum()
b = z[['hap1_n','hap2_n']]
#v = pd.merge(new_list,h2,how ='left', on=None, left_on=['pbID'], right_on=['pbID']
b

,hap1_n,hap2_n
pbID,,
PB.1000,957,1102
PB.10208,751,851
PB.10215,20,24
PB.10217,224,318
PB.10218,376,389
...,...,...
PB.9852,24,20
PB.9858,39,38
PB.9927,25,30


In [10]:
drop_id = b[b.sum(axis =1) == 0].index
b = b[~b.index.isin(drop_id)]


In [11]:
from scipy.stats import binomtest
results = b.apply(lambda x: binomtest(k=x.iloc[0], n = x.iloc[0] + x.iloc[1]), axis = 1)

b['p'] = results.apply(lambda x: x.pvalue)
b['conf_low'] = results.apply(lambda x : x.proportion_ci(confidence_level = .95)[0])
b['conf_high'] = results.apply(lambda x : x.proportion_ci(confidence_level = .95)[1])


##Benjamini-Hochberg Equation 

# (i/m)*Q
# i = rank of p-value
# m = total number of tests
# Q = your chosen false discovery rate

b = b.sort_values(by=['p'])

#b

In [12]:

FDR = []
prob = list(b['p'])
rank =1 

len_p_val_list = len(prob)

for p in prob:
    fdr_adj_p_val = p*len_p_val_list/rank
    rank += 1
    FDR.append(
        {
            "p_val": p,
            "fdr_adj_p_val": fdr_adj_p_val
        }
    )
#prob

In [13]:
b['Final_Prob'] = prob
b.to_csv('AH4.final.table.Muscle.Long')

In [14]:
b

,hap1_n,hap2_n,p,conf_low,conf_high,Final_Prob
pbID,,,,,,
PB.19125,2251,81,0.000000e+00,0.957012,0.972322,0.000000e+00
PB.23292,41037,763,0.000000e+00,0.980417,0.983008,0.000000e+00
PB.4069,2158,5859,0.000000e+00,0.259492,0.279032,0.000000e+00
PB.17899,7,501,4.022198e-138,0.005558,0.028184,4.022198e-138
PB.3813,432,2,4.255646e-126,0.983453,0.999441,4.255646e-126
...,...,...,...,...,...,...
PB.22073,1,0,1.000000e+00,0.025000,1.000000,1.000000e+00
PB.22150,79,79,1.000000e+00,0.419524,0.580476,1.000000e+00
PB.1588,18,19,1.000000e+00,0.319214,0.656003,1.000000e+00
